In [2]:
# most of our imports

from bloqade import rydberg_h, piecewise_linear, piecewise_constant, waveform, cast
from bloqade.atom_arrangement import ListOfLocations, Lieb, Square, Chain, Honeycomb, Kagome, Triangular, Rectangular
from bokeh.io import output_notebook # to plot "show()" on the notebook, without opening a browser
import os
import matplotlib.pyplot as plt
import numpy as np

output_notebook()

Loading BokehJS ...

In [16]:
# coordinates of our atoms in the array

#atom_coordinate = ListOfLocations([(21, 22), (21, 26), (21, 30), (21, 38), (21, 42), (21, 46), (21, 50), (21, 54), (25, 22), (25, 26), (25, 30), (25, 34), (25, 46), (25, 50), (25, 54), (29, 26), (29, 30), (29, 34), (29, 38), (29, 46), (29, 50), (29, 54), (33, 22), (33, 26), (33, 30), (33, 38), (33, 42), (33, 50), (33, 54), (37, 30), (37, 42), (37, 46), (37, 54), (41, 26), (41, 30), (41, 34), (41, 38), (41, 46), (41, 50), (41, 54), (45, 30), (45, 34), (45, 42), (45, 50), (45, 54), (49, 22), (49, 26), (49, 30), (49, 38), (49, 42), (53, 22), (53, 26), (53, 30), (53, 34), (53, 38), (53, 42), (53, 54)])
atom_coordinate = ListOfLocations([(31.5,32),(31.5,36),(31.5,44),(35.5,32),(35.5,40),(39.5,32),(39.5,36),(39.5,40),(39.5,44),(43.5,32),(43.5,36)])

In [17]:
# constants
rb = np.sqrt(3*np.sqrt(10))*4
C = 2*np.pi*862690
delta_end = (C / (rb**6))
delta_start = -delta_end
omega_max = 6
print(delta_end)
print(rb)

# # pulse waves

# durations_omega = [0.15] + [0.1 for _ in range(37)] + [0.1] + [0.15]

durations_delta = [0.15,3.15,0.7]

# durations_omega = [0.15] + [0.15]*24 + [0.15] + [0.1]
#durations_omega = [0.1 for _ in range (40)]

durations_omega = [0.15,3.7,0.15]

omega_MHz = [0,omega_max,omega_max,0]

#omega_MHz = [0] + ([omega_max]+[omega_max]+[0.1])*8 + [omega_max] + [omega_max] + [0]

delta_MHz=[delta_start,delta_start,delta_end,delta_end]

Delta = piecewise_linear(durations_delta,[x for x in delta_MHz])
Omega = piecewise_linear(durations_omega,[x for x in omega_MHz])

#create Hamiltonian
program = rydberg_h(atom_coordinate, detuning= Delta, amplitude=Omega, phase=None)

# display compiled atom array and pulses
program.parse_register().show()
program.parse_sequence().show()

1.5499258447301134
12.320281152964093


In [18]:
# run our algorithm!

output_6 = program.braket.aquila().run_async(shots=100)

In [19]:
# fetch our results and display

output_6.fetch()
output_6.report().show()

In [22]:
# report bit strings of outcome measurements

bit_strings = output_6.report().bitstrings()

11

In [23]:
from itertools import combinations
import matplotlib.pyplot as plt

def euclidean_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

n=57 # Number of atoms in array

# points: an array of coordinates of all vertices (atoms)

#points = [(21, 22), (21, 26), (21, 30), (21, 38), (21, 42), (21, 46), (21, 50), (21, 54), (25, 22), (25, 26), (25, 30), (25, 34), (25, 46), (25, 50), (25, 54), (29, 26), (29, 30), (29, 34), (29, 38), (29, 46), (29, 50), (29, 54), (33, 22), (33, 26), (33, 30), (33, 38), (33, 42), (33, 50), (33, 54), (37, 30), (37, 42), (37, 46), (37, 54), (41, 26), (41, 30), (41, 34), (41, 38), (41, 46), (41, 50), (41, 54), (45, 30), (45, 34), (45, 42), (45, 50), (45, 54), (49, 22), (49, 26), (49, 30), (49, 38), (49, 42), (53, 22), (53, 26), (53, 30), (53, 34), (53, 38), (53, 42), (53, 54)]
points = [(31.5,32),(31.5,36),(31.5,44),(35.5,32),(35.5,40),(39.5,32),(39.5,36),(39.5,40),(39.5,44),(43.5,32),(43.5,36)]

valid_subsets = []
lMIS=0

def ISR(n,bit_strings):
    '''
    Parameters:
    n: the number of qubits in your graph
    bit_strings: the bit strings returned from hardware of measured outcome states
    '''
    
    for i in range(0,len(bit_strings[0])):
        tracker = []
        for j in range(0,11):
            if bit_strings[0][i][j] == 0:
                tracker.append(j)
        new_points = [points[point_counter] for point_counter in tracker]
        for r in range(len(tracker),0,-1):
            combs = list(combinations(new_points, r))
            combs_as_lists = [list(comb) for comb in combs]
            t = False
            for subset in combs_as_lists:
                if all(euclidean_distance(subset[a], subset[b]) > 12 for a in range(len(subset)) for b in range(a + 1, len(subset))):
                    valid_subsets.append(subset)
                    t = True
            if t:
                lMIS =+ 1
            break

    print(lMIS)

1
